In [4]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\itiaw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
data_path = "../data/txt/"

# Choix d'une décennie et du nombre de clusters

In [32]:
DECADE = '1880'
N_CLUSTERS = 5

# Chargement des fichiers de la décennie

In [33]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [34]:
# Exemple de fichiers
files[:5]

['Bxl_1880_Tome_I1_Part_1.txt',
 'Bxl_1880_Tome_I1_Part_10.txt',
 'Bxl_1880_Tome_I1_Part_2.txt',
 'Bxl_1880_Tome_I1_Part_3.txt',
 'Bxl_1880_Tome_I1_Part_4.txt']

In [41]:
texts = [open(data_path + f, encoding='utf8').read() for f in files]

In [44]:
# Exemple de textes
texts[0][:400]

'VILLE\n\nD E\n\nBULLETIN\n\nBRUXELLES.\n\nCOMMUNAL\n\nA N N É E\n\n1880.\n\nPREMIÈRE PARTIE.\n\nCOMPTE\n\nRENDU\n\nDES\n\nSÉANCES.\n\nBBUXELLES,\n\nIMPRIMERIE DE V JULIEN BAERTSOEN,\nGrand\'Place, 5.\ne\n\n\x0c]\n\nI\n\n£\n\ni\n\n!\n\n\x0cN " 1.\n\nCOMPTE RENDU DE LA SÉANCE DÛ 1 2 JANVIER 1 8 8 0 .\n\nVILLE\n\nDE\n\nBRUXELLES.\n\nBULLETIN\n\nC O M M U N A L\n\nANNÉE\n\nC O N S E I L\n\n1880.\n\nC O M M U N A L .\n\nSéance du 42 Janvier 1880.\nPrésidence de M . F . V'

# Vectorisation du texte

In [48]:
def process_text(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [51]:
nltk.download('stopwords')
vectorizer = TfidfVectorizer(tokenizer=process_text,
                                stop_words=stopwords.words('french'),
                                max_df=0.5,
                                min_df=0.1,
                                lowercase=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\itiaw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
%time tfidf_vectors = vectorizer.fit_transform(texts)

Wall time: 1min 59s


In [53]:
tfidf_vectors

<270x7997 sparse matrix of type '<class 'numpy.float64'>'
	with 461792 stored elements in Compressed Sparse Row format>

In [54]:
# Exemple de vecteur TFIDF
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names()
    ).sort_values(ascending=False)

delecosse        0.338644
délégation       0.206433
trappeniers      0.199561
veldekens        0.179754
monte-charges    0.172714
                   ...   
lout             0.000000
louée            0.000000
louées           0.000000
loués            0.000000
#                0.000000
Length: 7997, dtype: float64

# Comprendre les vecteurs et leurs "distances"

In [55]:
from scipy.spatial.distance import cosine

In [56]:
cosine([1, 2, 3], [1, 2, 3])

0.0

In [57]:
cosine([1, 2, 3], [1, 2, 2])

0.02004211298777725

In [58]:
cosine([1, 2, 3], [2, 2, 2])

0.07417990022744858

In [59]:
tfidf_array = tfidf_vectors.toarray()

In [60]:
tfidf_array[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [61]:
texts[0][:200]

'VILLE\n\nD E\n\nBULLETIN\n\nBRUXELLES.\n\nCOMMUNAL\n\nA N N É E\n\n1880.\n\nPREMIÈRE PARTIE.\n\nCOMPTE\n\nRENDU\n\nDES\n\nSÉANCES.\n\nBBUXELLES,\n\nIMPRIMERIE DE V JULIEN BAERTSOEN,\nGrand\'Place, 5.\ne\n\n\x0c]\n\nI\n\n£\n\ni\n\n!\n\n\x0cN " 1.\n\n'

In [62]:
tfidf_array[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [63]:
cosine(tfidf_array[0], tfidf_array[1])

0.9012493988104938

# Clustering des vecteurs TFIDF

Article intéressant sur le KMeans clustering :
- https://medium.com/dataseries/k-means-clustering-explained-visually-in-5-minutes-b900cc69d175

In [64]:
km_model = KMeans(n_clusters=N_CLUSTERS)

In [65]:
km_model.fit(tfidf_vectors)

KMeans(n_clusters=5)

In [67]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(km_model.labels_):
    clustering[label].append(files[idx])

In [68]:
pprint(dict(clustering))

{0: ['Bxl_1880_Tome_I1_Part_1.txt',
     'Bxl_1880_Tome_I1_Part_10.txt',
     'Bxl_1880_Tome_I1_Part_2.txt',
     'Bxl_1880_Tome_I1_Part_3.txt',
     'Bxl_1880_Tome_I1_Part_4.txt',
     'Bxl_1880_Tome_I1_Part_5.txt',
     'Bxl_1880_Tome_I1_Part_6.txt',
     'Bxl_1880_Tome_I1_Part_7.txt',
     'Bxl_1880_Tome_I1_Part_8.txt',
     'Bxl_1880_Tome_I1_Part_9.txt',
     'Bxl_1881_Tome_I1_Part_1.txt',
     'Bxl_1881_Tome_I1_Part_10.txt',
     'Bxl_1881_Tome_I1_Part_2.txt',
     'Bxl_1881_Tome_I1_Part_3.txt',
     'Bxl_1881_Tome_I1_Part_4.txt',
     'Bxl_1881_Tome_I1_Part_5.txt',
     'Bxl_1881_Tome_I1_Part_6.txt',
     'Bxl_1881_Tome_I1_Part_7.txt',
     'Bxl_1881_Tome_I1_Part_8.txt',
     'Bxl_1881_Tome_I1_Part_9.txt',
     'Bxl_1881_Tome_I2_Part_1.txt',
     'Bxl_1881_Tome_I2_Part_10.txt',
     'Bxl_1881_Tome_I2_Part_11.txt',
     'Bxl_1881_Tome_I2_Part_12.txt',
     'Bxl_1881_Tome_I2_Part_2.txt',
     'Bxl_1881_Tome_I2_Part_3.txt',
     'Bxl_1881_Tome_I2_Part_4.txt',
     'Bxl_1881_Tome_I2_